In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd

df_train = pd.read_csv('train_kaggle.csv')
df_test = pd.read_csv('sample_solution.csv')
y = df_train['label']
y = y.values

In [10]:
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier

NUM_OF_COL = 40

def load_features(col):
    return pd.read_parquet("filtered_features/filtered_feature{}.gzip".format(col))

def load_test_features(col):
    return pd.read_parquet("filtered_test_features/filtered_test_feature{}.gzip".format(col))

def load_all_features(col,y):
    train = load_features(col)
    test = load_test_features(col).replace([np.inf], np.finfo('float32').max).replace([np.inf, -np.inf], np.finfo('float32').min).fillna(0)
    m = get_feature_model(train, y)
    return m.transform(train), m.transform(test)

def select_all(selected_features, y):
    train = []
    test = []
    for col in tqdm(range(NUM_OF_COL)):
        if selected_features[col] == False:
            continue
        tr, te = load_all_features(col, y)
        train.append(tr)
        test.append(te)
    train = np.concatenate(train, axis=1)
    test = np.concatenate(test, axis=1)
    return train, test

def select_train(selected_features):
    train = []
    for col in tqdm(range(NUM_OF_COL)):
        if selected_features[col] == False:
            continue
        tf = load_features(col)
        train.append(tf)
    train = np.concatenate(train,  axis=1)
    return train

def select_test(selected_features):
    test = []
    for col in range(NUM_OF_COL):
        if selected_features[col] == False:
            continue
        tf = load_test_features(col)
        tf = tf.replace([np.inf], np.finfo('float32').max).replace([np.inf, -np.inf], np.finfo('float32').min).fillna(0)
        test.append(tf)
    test = np.concatenate(test,  axis=1)
    return test

def calculate_auc(train, y):
    X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size=0.2, random_state=42)
    model = LGBMClassifier()
    model.fit(X_train, y_train)
    pred = model.predict_proba(X_valid)[:,1]
    fpr, tpr, thresholds = metrics.roc_curve(y_valid, pred)
    auc_score = metrics.auc(fpr, tpr)
    return auc_score

def show_index(selected_features):
    return np.where(np.array(selected_features) == False)

def get_extra_tree(train, y):
    clf = ExtraTreesClassifier(n_estimators=50)
    clf = clf.fit(train, y)
    return clf

def get_feature_model(train, y, threshold=None):
    clf = get_extra_tree(train, y)
    # print(clf.feature_importances_)
    modelSelection = SelectFromModel(clf, prefit=True, max_features=8000, threshold=threshold)
    return modelSelection

selectall

In [11]:
import numpy as np

selected_features = [ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, True,  True,  True,  True,  True,  True, True,  True,
        True,  True,  True,  True,  True,  True,  True, True,  True,
        False,  True,  True,  True,  True,  True,  True, True,  True,
        True,  True,  True,  True]
print(show_index(selected_features))
train, test = select_all(selected_features,y)
print(train.shape)

(array([27], dtype=int64),)



100%|█████████████████████████████████████████████████████████| 40/40 [05:26<00:00,  8.16s/it]


(30336, 3923)


In [12]:
calculate_auc(train, y)

0.9562132463580377

## Frequency importance

In [9]:
import numpy as np

selected_features = [ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        False,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True]
print(show_index(selected_features))
train = select_train(selected_features)
print(train.shape)

(array([10, 16, 25, 27, 34], dtype=int64),)



100%|█████████████████████████████████████████████████████████| 40/40 [00:10<00:00,  3.67it/s]


(30336, 12518)


In [ ]:
calculate_auc(train, y)
# 0.9559280694365483

In [33]:
modelSelection = get_feature_model(train, y)
train_trans = modelSelection.transform(train)
calculate_auc(train_trans, y)
# 0.9531738028709031

100%|█████████████████████████████████████████████████████████| 40/40 [00:16<00:00,  2.50it/s]


[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.98439752e-05
 2.75254402e-05 4.96375509e-05]


0.9531738028709031

In [42]:
train_trans.shape

(30336, 2934)

In [36]:
clf = get_extra_tree(train, y)
clf.feature_importances_

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
       1.97294762e-05, 2.71300905e-05, 1.60803685e-05])

In [41]:
np.where(clf.feature_importances_ > 0.0001)[0].shape

(1704,)

## Some random trial

In [26]:
import numpy as np

selected_features = [ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, True,  True,  True,  True,  True,  True, True,  True,
        True,  True,  True,  True,  True,  True,  True, True,  True,
        False,  True,  True,  True,  True,  True,  True, True,  True,
        True,  True,  True,  True]
print(show_index(selected_features))
train = select_train(selected_features)
print(train.shape)

(array([27], dtype=int64),)


100%|█████████████████████████████████████████████████████████| 40/40 [00:12<00:00,  3.29it/s]


(30336, 14288)


0.9578389014310529

In [ ]:
calculate_auc(train, y)
# 0.9578389014310529

In [31]:
modelSelection = get_feature_model(train, y)
train_trans = modelSelection.transform(train)
calculate_auc(train_trans, y)
# 0.9569397510603413

[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 5.14626514e-05
 2.03923644e-05 7.09797664e-05]


0.9569397510603413

## Some random trial 2

In [55]:
import numpy as np

selected_features = [ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, True,  True,  True,  True,  True,  True, True,  True,
        True,  True,  True,  True,  False,  True,  True, True,  True,
        False,  True,  True,  True,  True,  True,  True, True,  True,
        True,  True,  True,  True]
print(show_index(selected_features))
train = select_train(selected_features)
print(train.shape)

(array([22, 27], dtype=int64),)


100%|█████████████████████████████████████████████████████████| 40/40 [00:18<00:00,  2.17it/s]


(30336, 13864)


In [56]:
calculate_auc(train, y)


0.9558481612503212

## Best Feature Group so far

In [4]:
import numpy as np

selected_features = [ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, True,  True,  True,  True,  True,  True, True,  True,
        False,  True,  True,  True,  True,  True,  True, True,  True,
        True,  True,  True,  True,  True,  True,  True, True,  True,
        True,  True,  True,  True]
print(show_index(selected_features))
train = select_train(selected_features)
print(train.shape)

(array([18], dtype=int64),)


100%|█████████████████████████████████████████████████████████| 40/40 [00:12<00:00,  3.12it/s]


(30336, 14136)


In [46]:
calculate_auc(train, y)
# 0.9547855289940256

0.9547855289940256

In [5]:
modelSelection = get_feature_model(train, y)
train_trans = modelSelection.transform(train)
calculate_auc(train_trans, y)
# 0.9561219750811085

[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 8.30697945e-05
 3.84732813e-05 3.13587033e-05]


0.9582666668132872

In [10]:
model = get_extra_tree(train,y)
np.where(model.feature_importances_ > np.mean(model.feature_importances_))

(array([   10,    12,    13, ..., 14124, 14126, 14128], dtype=int64),)

## Trial 3

In [48]:
import numpy as np

selected_features = [ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True, False,  True,
        False,  True,  True,  True,  True,  True,  True, False,  True,
        False,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True]
print(show_index(selected_features))
train = select_train(selected_features)
print(train.shape)

(array([10, 16, 18, 25, 27, 34], dtype=int64),)


100%|█████████████████████████████████████████████████████████| 40/40 [00:09<00:00,  4.14it/s]


(30336, 12090)


In [49]:
calculate_auc(train, y)

0.9549387474428463

## Trial 4

In [57]:
import numpy as np

selected_features = [ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, True,  True,  True,  False,  True,  True, True,  True,
        True,  True,  True,  True,  True,  True,  True, True,  True,
        False,  True,  True,  True,  True,  True,  True, True,  True,
        True,  True,  True,  True]
print(show_index(selected_features))
train = select_train(selected_features)
print(train.shape)

(array([13, 27], dtype=int64),)


100%|█████████████████████████████████████████████████████████| 40/40 [00:12<00:00,  3.10it/s]


(30336, 13821)


In [58]:
calculate_auc(train, y)

0.9508630633939499

## Trial 5

In [74]:
import numpy as np

selected_features = [ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, True,  True,  True,  True,  True,  True, True,  True,
        False,  True,  True,  True,  True,  True,  True, True,  True,
        False,  True,  True,  True,  True,  True,  True, True,  True,
        True,  True,  True,  True]
print(show_index(selected_features))
train = select_train(selected_features)
print(train.shape)

(array([18, 27], dtype=int64),)


100%|█████████████████████████████████████████████████████████| 40/40 [00:12<00:00,  3.27it/s]


(30336, 13860)


In [75]:
calculate_auc(train, y)

0.9558525598660769

In [76]:
modelSelection = get_feature_model(train, y)
train_trans = modelSelection.transform(train)
calculate_auc(train_trans, y)

[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.37663205e-05
 5.69949131e-05 4.03709044e-05]


0.9568598428741142

In [77]:
train_trans.shape

(30336, 3179)

## Frequency importance 2 

In [50]:
import numpy as np

selected_features = [False,  True,  True,  True, False, False,  True, False,  True,
       False, False,  True,  True,  True,  True,  True, False,  True,
        True, False,  True, False,  True,  True,  True, False, False,
       False, False,  True, False,  True,  True,  True, False, False,
       False,  True, False,  True]
print(show_index(selected_features))
train = select_train(selected_features)
print(train.shape)

(array([ 0,  4,  5,  7,  9, 10, 16, 19, 21, 25, 26, 27, 28, 30, 34, 35, 36,
       38], dtype=int64),)


100%|█████████████████████████████████████████████████████████| 40/40 [00:05<00:00,  6.69it/s]


(30336, 7396)


In [51]:
calculate_auc(train, y)

0.9523124072854272

## Frequency importance 3

In [83]:
import numpy as np

selected_features = [ True,  True, False, False,  True, False,  True,  True,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True,  True,  True, False,  True,  True, False,  True, False,
       False,  True,  True,  True,  True, False,  True,  True, False,
       False, False,  True,  True]
print(show_index(selected_features))
train = select_train(selected_features)
print(train.shape)

(array([ 2,  3,  5,  9, 11, 13, 15, 17, 21, 24, 26, 27, 32, 35, 36, 37],
      dtype=int64),)


100%|█████████████████████████████████████████████████████████| 40/40 [00:08<00:00,  4.67it/s]


(30336, 9020)


In [84]:
calculate_auc(train, y)

0.9394486555081116

## Real Training

### Training with untuned

In [85]:
selected_features = [ True,  True, False, False,  True, False,  True,  True,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True,  True,  True, False,  True,  True, False,  True, False,
       False,  True,  True,  True,  True, False,  True,  True, False,
       False, False,  True,  True]

XTrain = select_train(selected_features)
XTest = select_test(selected_features)

100%|█████████████████████████████████████████████████████████| 40/40 [00:08<00:00,  4.63it/s]


In [86]:
modelSelection = get_feature_model(XTrain, y)
XTrain = modelSelection.transform(XTrain)
XTest = modelSelection.transform(XTest)

[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 7.11317099e-05
 5.76997440e-05 6.73268251e-05]


In [17]:
selected_features

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [13]:
XTrain, XTest = select_all(selected_features, y)


100%|█████████████████████████████████████████████████████████| 40/40 [05:24<00:00,  8.12s/it]


In [14]:
print(XTrain.shape)
print(XTest.shape)

(30336, 3903)
(10000, 3903)


In [15]:
model = LGBMClassifier()
model.fit(XTrain, y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [16]:
probs = model.predict_proba(XTest)
YTest = probs[:, 0]
df_test['Predicted'] = YTest
df_test.to_csv('test.csv', index=False)
df_test

,Id,Predicted
0,0,0.995233
1,1,0.996870
2,2,0.975159
3,3,0.995921
4,4,0.997603
5,5,0.985117
6,6,0.981689
7,7,0.925348
8,8,0.997823
9,9,0.996330


### Split

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(XTrain, y, test_size=0.2, random_state=42)

In [19]:
import lightgbm as lgb

train_set = lgb.Dataset(X_train, y_train)
valid_set = lgb.Dataset(X_valid, y_valid)

In [20]:
param = {'num_leaves': 31, 'objective': 'binary'}
param['metric'] = 'auc'
bst = lgb.train(param, train_set, 5, valid_sets=valid_set, early_stopping_rounds=5)
bst.best_iteration 

[1]	valid_0's auc: 0.864314
Training until validation scores don't improve for 5 rounds
[2]	valid_0's auc: 0.886711
[3]	valid_0's auc: 0.893939
[4]	valid_0's auc: 0.911327
[5]	valid_0's auc: 0.91446
Did not meet early stopping. Best iteration is:
[5]	valid_0's auc: 0.91446


5

In [21]:
bst.best_score['valid_0']['auc']

0.9144597528491117

In [22]:
import lightgbm as lgb
from hyperopt import STATUS_OK

ITER = 50
STOP_ROUND = 5

# Create the dataset
def objective(params):
    """Objective function for Gradient Boosting Machine Hyperparameter Tuning"""
    params['num_leaves'] = int(params['num_leaves'])
    params['subsample_for_bin'] = int(params['subsample_for_bin'])
    params['min_child_samples'] = int(params['min_child_samples'])
    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evalute based on ROC AUC
    bst = lgb.train(params, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)
    bst.save_model('model.txt', num_iteration=bst.best_iteration)
  
    # Extract the best score
    best_score = bst.best_score['valid_0']['auc']
    
    # Loss must be minimized
    loss = np.log(1 - best_score)
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

In [23]:
from hyperopt import hp

# Define the search space
space = {
    'boosting_type': 'dart',
    'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    'metric': 'auc'
}

space

{'boosting_type': 'dart',
 'num_leaves': <hyperopt.pyll.base.Apply at 0x23f2ecc84a8>,
 'learning_rate': <hyperopt.pyll.base.Apply at 0x23f2ecc8588>,
 'subsample_for_bin': <hyperopt.pyll.base.Apply at 0x23f336689b0>,
 'min_child_samples': <hyperopt.pyll.base.Apply at 0x23f33668470>,
 'reg_alpha': <hyperopt.pyll.base.Apply at 0x23f05502828>,
 'reg_lambda': <hyperopt.pyll.base.Apply at 0x23f05502978>,
 'colsample_bytree': <hyperopt.pyll.base.Apply at 0x23f05502ac8>,
 'metric': 'auc'}

In [24]:
from hyperopt import Trials
# Trials object to track progress
bayes_trials = Trials()

In [ ]:
from hyperopt import fmin
from hyperopt import tpe

MAX_EVALS = 500

# Optimize
bestDict = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials)
bestDict


  0%|                                                   | 0/500 [00:00<?, ?it/s, best loss: ?]
[1]	valid_0's auc: 0.905277                                                                   

  0%|                                                   | 0/500 [00:02<?, ?it/s, best loss: ?]

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\callback.py:192: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')

  0%|                                                                | 0/40 [2:21:00<?, ?it/s]


[2]	valid_0's auc: 0.912479                                                                   

  0%|                                                   | 0/500 [00:05<?, ?it/s, best loss: ?]
[3]	valid_0's auc: 0.922163                                                                   

  0%|                                                   | 0/500 [00:07<?, ?it/s, best loss: ?]
[4]	valid_0's auc: 0.924853                                                                   

  0%|                                                   | 0/500 [00:09<?, ?it/s, best loss: ?]
[5]	valid_0's auc: 0.928851                                                                   

  0%|                                                   | 0/500 [00:11<?, ?it/s, best loss: ?]
[6]	valid_0's auc: 0.928468                                                                   

  0%|                                                   | 0/500 [00:13<?, ?it/s, best loss: ?]
[7]	valid_0's auc: 0.92986                  

  0%|                                                   | 0/500 [01:13<?, ?it/s, best loss: ?]
[45]	valid_0's auc: 0.940526                                                                  

  0%|                                                   | 0/500 [01:15<?, ?it/s, best loss: ?]
[46]	valid_0's auc: 0.940427                                                                  

  0%|                                                   | 0/500 [01:16<?, ?it/s, best loss: ?]
[47]	valid_0's auc: 0.940289                                                                  

  0%|                                                   | 0/500 [01:18<?, ?it/s, best loss: ?]
[48]	valid_0's auc: 0.940147                                                                  

  0%|                                                   | 0/500 [01:19<?, ?it/s, best loss: ?]
[49]	valid_0's auc: 0.940393                                                                  

  0%|                                        

  0%|                      | 1/500 [01:37<11:26:14, 82.51s/it, best loss: -2.8194025631870363]
[37]	valid_0's auc: 0.942362                                                                  

  0%|                      | 1/500 [01:38<11:26:14, 82.51s/it, best loss: -2.8194025631870363]
[38]	valid_0's auc: 0.942415                                                                  

  0%|                      | 1/500 [01:38<11:26:14, 82.51s/it, best loss: -2.8194025631870363]
[39]	valid_0's auc: 0.942234                                                                  

  0%|                      | 1/500 [01:39<11:26:14, 82.51s/it, best loss: -2.8194025631870363]
[40]	valid_0's auc: 0.942225                                                                  

  0%|                      | 1/500 [01:39<11:26:14, 82.51s/it, best loss: -2.8194025631870363]
[41]	valid_0's auc: 0.94229                                                                   

  0%|                      | 1/500 [01:39<11:

...

 20%|████▏                | 100/500 [52:06<4:18:19, 38.75s/it, best loss: -2.9974532457866903]
[10]	valid_0's auc: 0.933537                                                                  

 20%|████▏                | 100/500 [52:06<4:18:19, 38.75s/it, best loss: -2.9974532457866903]
[11]	valid_0's auc: 0.933792                                                                  

 20%|████▏                | 100/500 [52:07<4:18:19, 38.75s/it, best loss: -2.9974532457866903]
[12]	valid_0's auc: 0.934309                                                                  

 20%|████▏                | 100/500 [52:07<4:18:19, 38.75s/it, best loss: -2.9974532457866903]
[13]	valid_0's auc: 0.935001                                                                  

 20%|████▏                | 100/500 [52:07<4:18:19, 38.75s/it, best loss: -2.9974532457866903]
[14]	valid_0's auc: 0.934559                                                                  

 20%|████▏                | 100/500 [52:08<4:

 20%|████▏                | 101/500 [52:26<3:44:51, 33.81s/it, best loss: -2.9974532457866903]
[2]	valid_0's auc: 0.92265                                                                    

 20%|████▏                | 101/500 [52:27<3:44:51, 33.81s/it, best loss: -2.9974532457866903]
[3]	valid_0's auc: 0.925475                                                                   

 20%|████▏                | 101/500 [52:27<3:44:51, 33.81s/it, best loss: -2.9974532457866903]
[4]	valid_0's auc: 0.928162                                                                   

 20%|████▏                | 101/500 [52:27<3:44:51, 33.81s/it, best loss: -2.9974532457866903]
[5]	valid_0's auc: 0.929774                                                                   

 20%|████▏                | 101/500 [52:28<3:44:51, 33.81s/it, best loss: -2.9974532457866903]
[6]	valid_0's auc: 0.931873                                                                   

 20%|████▏                | 101/500 [52:29<3:

[44]	valid_0's auc: 0.943944                                                                  

 20%|████▏                | 101/500 [52:55<3:44:51, 33.81s/it, best loss: -2.9974532457866903]
[45]	valid_0's auc: 0.944439                                                                  

 20%|████▏                | 101/500 [52:56<3:44:51, 33.81s/it, best loss: -2.9974532457866903]
[46]	valid_0's auc: 0.944606                                                                  

 20%|████▏                | 101/500 [52:56<3:44:51, 33.81s/it, best loss: -2.9974532457866903]
[47]	valid_0's auc: 0.945379                                                                  

 20%|████▏                | 101/500 [52:57<3:44:51, 33.81s/it, best loss: -2.9974532457866903]
[48]	valid_0's auc: 0.945302                                                                  

 20%|████▏                | 101/500 [52:57<3:44:51, 33.81s/it, best loss: -2.9974532457866903]
[49]	valid_0's auc: 0.945619                 

[36]	valid_0's auc: 0.939629                                                                  

 20%|████▎                | 102/500 [53:17<3:44:56, 33.91s/it, best loss: -2.9974532457866903]
[37]	valid_0's auc: 0.939792                                                                  

 20%|████▎                | 102/500 [53:17<3:44:56, 33.91s/it, best loss: -2.9974532457866903]
[38]	valid_0's auc: 0.939706                                                                  

 20%|████▎                | 102/500 [53:18<3:44:56, 33.91s/it, best loss: -2.9974532457866903]
[39]	valid_0's auc: 0.939816                                                                  

 20%|████▎                | 102/500 [53:18<3:44:56, 33.91s/it, best loss: -2.9974532457866903]
[40]	valid_0's auc: 0.939978                                                                  

 20%|████▎                | 102/500 [53:19<3:44:56, 33.91s/it, best loss: -2.9974532457866903]
[41]	valid_0's auc: 0.939939                 

[28]	valid_0's auc: 0.935587                                                                  

 21%|████▎                | 103/500 [53:41<3:30:18, 31.78s/it, best loss: -2.9974532457866903]
[29]	valid_0's auc: 0.936348                                                                  

 21%|████▎                | 103/500 [53:41<3:30:18, 31.78s/it, best loss: -2.9974532457866903]
[30]	valid_0's auc: 0.937102                                                                  

 21%|████▎                | 103/500 [53:42<3:30:18, 31.78s/it, best loss: -2.9974532457866903]
[31]	valid_0's auc: 0.937153                                                                  

 21%|████▎                | 103/500 [53:42<3:30:18, 31.78s/it, best loss: -2.9974532457866903]
[32]	valid_0's auc: 0.937402                                                                  

 21%|████▎                | 103/500 [53:43<3:30:18, 31.78s/it, best loss: -2.9974532457866903]
[33]	valid_0's auc: 0.938422                 

In [ ]:
from hyperopt import space_eval
best = space_eval(space, bestDict)
best['num_leaves'] = int(best['num_leaves'])
best['subsample_for_bin'] = int(best['subsample_for_bin'])
best['min_child_samples'] = int(best['min_child_samples'])
best['metric'] = 'auc'
bestModel = lgb.train(best, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)

In [ ]:
bestModel.save_model('model.txt', num_iteration=bestModel.best_iteration)

In [ ]:
bestModel.num_trees()

In [ ]:
best

In [ ]:
from lightgbm import LGBMClassifier

gb_clf = LGBMClassifier(boosting_type=best['boosting_type'],
                        num_leaves=best['num_leaves'],
                        learning_rate=best['learning_rate'],
                        subsample_for_bin=best['subsample_for_bin'],
                        min_child_samples=best['min_child_samples'],
                        reg_alpha=best['reg_alpha'],
                        reg_lambda=best['reg_lambda'],
                        colsample_bytree=best['colsample_bytree'])
gb_clf.fit(XTrain, y)

In [ ]:
probs = gb_clf.predict_proba(XTest, num_iteration=bestModel.best_iteration)
YTest = probs[:, 1]
YTest

In [ ]:
np.max(YTest)

In [ ]:
df_test['Predicted'] = YTest
df_test

In [ ]:
df_test.to_csv('test.csv', index=False)

In [ ]:
from lightgbm import LGBMModel
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

clf = LGBMModel()
parameters = {'learning_rate': sp_randFloat(),
                  'subsample'    : sp_randFloat(),
                  'n_estimators' : sp_randInt(100, 1000),
                  'max_depth'    : sp_randInt(4, 10) 
                 }

randm = RandomizedSearchCV(estimator=gb_clf, param_distributions = parameters, 
                           cv = 5, n_iter = 10, n_jobs=-1)
randm.fit(XTrain, y)

# Results from Random Search
print("\n========================================================")
print(" Results from Random Search " )
print("========================================================")    

print("\n The best estimator across ALL searched params:\n",
      randm.best_estimator_)

print("\n The best score across ALL searched params:\n",
      randm.best_score_)

print("\n The best parameters across ALL searched params:\n",
      randm.best_params_)

print("\n ========================================================")

gb_clf = randm.best_estimator_